# Web scraping AirBnB

In [1]:
!cd /tmp/
!sudo wget https://chromedriver.storage.googleapis.com/111.0.5563.64/chromedriver_linux64.zip
!sudo unzip chromedriver_linux64.zip
!sudo mv chromedriver /usr/bin/chromedriver
!chromedriver --version

--2023-03-31 18:32:31--  https://chromedriver.storage.googleapis.com/111.0.5563.64/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 142.250.185.144, 2a00:1450:4001:810::2010
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|142.250.185.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7160988 (6.8M) [application/zip]
Saving to: ‘chromedriver_linux64.zip.1’

100%[======================================>] 7,160,988   --.-K/s   in 0.07s   

2023-03-31 18:32:31 (99.8 MB/s) - ‘chromedriver_linux64.zip.1’ saved [7160988/7160988]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
replace LICENSE.chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
ChromeDriver 111.0.5563.64 (c710e93d5b63b7095afe8c2c17df34408078439d-refs/branch-heads/5563@{#995})


In [2]:
!sudo curl https://intoli.com/install-google-chrome.sh | bash
!sudo mv /usr/bin/google-chrome-stable /usr/bin/google-chrome
!google-chrome --version && which google-chrome

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9526  100  9526    0     0  19278      0 --:--:-- --:--:-- --:--:-- 19283
Working in /tmp/google-chrome-installation
/tmp/google-chrome-installation /home/ec2-user/SageMaker
Configuring the Google Chrome repo in /etc/yum.repos.d/google-chrome.repo
Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.7 kB     00:00     
amzn2extra-docker                                        | 3.0 kB     00:00     
amzn2extra-kernel-5.10                                   | 3.0 kB     00:00     
amzn2extra-python3.8                                     | 3.0 kB     00:00     
centos-extras                                            | 2.9 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 

In [3]:
%pip install selenium webdriver_manager bs4

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 7.1 MB/s eta 0:00:000:00:010:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 15.3 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=c0267da17b0942ef59c76ad1d7b1361086f425f1823c856d661cecdad39fc468
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e4/62/1d/d4d1bc4f33350ff84227f89b258edb552d604138e3739f5c83
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import time
import pandas as pd
from pandas import DataFrame
import urllib.parse as urlparse
from urllib.parse import parse_qs

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

# Get current working directory
print(os.getcwd())

/home/ec2-user/SageMaker


# Urls to scrape

In [2]:
# urlStringHome = 'https://www.airbnb.ch/'
# urlString = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt"
# urlStringFlat0to68 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=9&price_max=68"
# urlStringFlat69to84 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=69&price_max=84"
# urlStringFlat85to100 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=85&price_max=100"
# urlStringFlat101to114 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=101&price_max=114"
# urlStringFlat115to132 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=115&price_max=132"
# urlStringFlat133to150 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=133&price_max=150"
# urlStringFlat151to173 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=151&price_max=173"
# urlStringFlat174to202 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=151&price_max=173"
# urlStringFlat203to245 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=203&price_max=245"
# urlStringFlat246to316 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=246&price_max=316"
# urlStringFlat317to500 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=317&price_max=500"
# urlStringFlat501to1000 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=501&price_max=1000"
# define price range for scraping, place etc is already definied in buildURL func
url_range = [[0,68],[69,84],[85,100],[101,114],[114,132],[133,150],[151,173],[174,202],[203,245],[246,316],[317,500],[501,1000]]

In [3]:
def buildURL(p_range):
    builded_urls = []
    for p in p_range:
        url = f"https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min={p[0]}&price_max={p[1]}"
        builded_urls.append(url)
    return builded_urls

## Fields to scrape from detail apartment page

* Preis
* Standort
* Gesamt Rating
  * Alle Subfelder Sauberkeit, Kommunikation etc.
* Anz. Gäste
* Anz. Schlafzimmer 
* Anz. Better
* Anz. Badezimmer 
* Ameneties
* Koordinaten Wohnung
* Anz. Bewertungen
* Kommentare

## Helper functions for scraping

In [4]:
# helper func to scrape comments
def scrape_comments(driver):
    comments = []
    XPATH_COMMENTS_BTN = "//button[contains(@aria-label, 'Bewertungen anzeigen, Öffnet den Modaldialog.')]"
    XPATH_COMMENTS_BTN_CLOSE = "//button[@aria-label='Schließen']"
    XPATH_COMMENTS_CLASS = "span[class='ll4r2nl dir dir-ltr']"
    XPATH_COMMENTS_ON_SITE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[3]/div/div/*"

    try:
        WebDriverWait(driver, 4).until(EC.visibility_of_element_located(
            (By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]")))
    except:
        print("could not load comments title")
        return comments

    try:
        if driver.find_elements(By.XPATH, XPATH_COMMENTS_BTN):
            driver.find_element(By.XPATH, XPATH_COMMENTS_BTN).click()
            WebDriverWait(driver, 4).until(EC.visibility_of_element_located(
                (By.XPATH, "//div[contains(@data-testid, 'pdp-reviews-modal-scrollable-panel')]")))
            resu = driver.find_elements(By.CSS_SELECTOR, XPATH_COMMENTS_CLASS)
            for comm in resu:
                if comm.text:
                    comments.append(comm.text)
            driver.find_element(By.XPATH, XPATH_COMMENTS_BTN_CLOSE).click()

        elif not driver.find_elements(By.XPATH, XPATH_COMMENTS_BTN) and len(driver.find_elements(By.XPATH, XPATH_COMMENTS_ON_SITE)) > 0:
            comments_on_page = driver.find_elements(
                By.XPATH, XPATH_COMMENTS_ON_SITE)
            for comm_on in comments_on_page:
                if comm_on.text:
                    temp_comm_on = comm_on.text.split("\n", maxsplit=2)[2:]
                    comments.append(temp_comm_on[0])

        else:
            return comments

    except:
        print("could not process comments")
        return comments

    return comments


In [5]:
# helper func to scrape coordinates
def scrape_coordinates(driver):
    lat = 0.0
    lng = 0.0

    try:
        element_to_be_present_co = WebDriverWait(driver, 4).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[5]/div/div/div/div[2]/section/div[1]/div/h2")))
        driver.execute_script("arguments[0].scrollIntoView();", element_to_be_present_co)

        time.sleep(1)

        coordinates_elem = WebDriverWait(driver, 4).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(@href, 'https://maps.google.com/maps?ll=')]")))
        coordinates_href = coordinates_elem.get_attribute('href')

        parsed_href = urlparse.urlparse(coordinates_href)
        coordinates = parse_qs(parsed_href.query)['ll'][0].split(',')
        lat = float(coordinates[0])
        lng = float(coordinates[1])
    except:
        print("Could not process coordinates")

    return lat, lng


In [6]:
# helper func to scrape ameneties
def scrape_ameneties(driver):

    ameneties_list = []

    try:
        SHOW_AMENETIES_CLASS = "button[class='l1j9v1wn b65jmrv v7aged4 dir dir-ltr']"
        driver.find_element(By.CSS_SELECTOR, SHOW_AMENETIES_CLASS).click()

        element_present_ame = EC.presence_of_element_located(
            (By.XPATH, "//div[@aria-label='Das bietet dir diese Unterkunft']"))
        WebDriverWait(driver, 6).until(element_present_ame)

        soup = BeautifulSoup(driver.page_source, features='html.parser')
        ameneties = soup.find_all("div", {"class": "t1dx2edb dir dir-ltr"})

        for a in ameneties:
            if "</span>" not in a.decode_contents():
                ameneties_list.append(a.decode_contents())

    except:
        print("Could not process ameneties")

    try:
        driver.find_element(
            By.XPATH, "//button[@aria-label='Schließen']").click()
    except:
        print("Could not close ameneties window")

    return ameneties_list


In [11]:
# scrapping logic for detail page apartment
def scrape_apartment(nr_of_flat, driver):

    flat = None

    XPATH_cleanliness = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[1]/div/div/div[2]/span"
    XPATH_communication = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[3]/div/div/div[2]/span"
    XPATH_checkin = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[5]/div/div/div[2]/span"
    XPATH_valueformoney = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[6]/div/div/div[2]/span"
    XPATH_location = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[4]/div/div/div[2]/span"
    XPATH_data_accuracy = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[2]/div/div/div[2]/span"
    rating_cleanliness = ""
    rating_communication = ""
    rating_checkin = ""
    rating_valueformoney = ""
    rating_location = ""
    rating_data_accuracy = ""

    XPATH_APARETMENT_NAME = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[1]/span/h1"
    apartment_name = ""

    XPATH_PRICE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[1]"
    apartment_price = ""
    XPATH_PRICE_REDUCED = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[2]"
    apartment_price_reduced = ""

    XPATH_PLACE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[3]/button/span"
    XPATH_PLACE_2 = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[5]/button/span"
    apartment_place = ""

    XPATH_RATING = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[1]/span[2]"
    rating = ""

    XPATH_GUESTS = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[1]/span[1]"
    guests = ""
    XPATH_BEDROOMS = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[2]/span[2]"
    bedrooms = ""
    XPATH_BEDS = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[3]/span[2]"
    beds = ""
    XPATH_BATHROOMS = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[4]/span[2]"
    bathrooms = ""

    XPATH_NR_OF_RATINGS = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[1]/span[3]/button"
    nr_of_ratings = ""

    # check if page title is present
    try:
        element_present = EC.presence_of_element_located(
            (By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[1]/span/h1"))
        WebDriverWait(driver, 8).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        return

    try:
        if driver.find_elements(By.XPATH, XPATH_PLACE):
            apartment_place = WebDriverWait(driver, 3).until(
                EC.visibility_of_element_located((By.XPATH, XPATH_PLACE))).text
            if "Zürich" not in apartment_place:
                return flat
        else:
            apartment_place = WebDriverWait(driver, 3).until(
                EC.visibility_of_element_located((By.XPATH, XPATH_PLACE_2))).text
            if "Zürich" not in apartment_place:
                return flat
    except:
        pass

    try:
        apartment_name = WebDriverWait(driver, 3).until(
            EC.visibility_of_element_located((By.XPATH, XPATH_APARETMENT_NAME))).text
        nr_of_ratings = WebDriverWait(driver, 4).until(
            EC.visibility_of_element_located((By.XPATH, XPATH_NR_OF_RATINGS))).text
    except:
        pass

    try:
        apartment_price = WebDriverWait(driver, 4).until(
            EC.visibility_of_element_located((By.XPATH, XPATH_PRICE))).text

        if driver.find_elements(By.XPATH, XPATH_PRICE_REDUCED):
            apartment_price_reduced = driver.find_element(
                By.XPATH, XPATH_PRICE_REDUCED).text
            if "CHF" not in apartment_price_reduced:
                apartment_price_reduced = ""
    except:
        pass

    try:
        rating = driver.find_element(By.XPATH, XPATH_RATING).text
        rating_checkin = driver.find_element(By.XPATH, XPATH_checkin).text
        rating_cleanliness = driver.find_element(
            By.XPATH, XPATH_cleanliness).text
        rating_communication = driver.find_element(
            By.XPATH, XPATH_communication).text
        rating_data_accuracy = driver.find_element(
            By.XPATH, XPATH_data_accuracy).text
        rating_location = driver.find_element(By.XPATH, XPATH_location).text
        rating_valueformoney = driver.find_element(
            By.XPATH, XPATH_valueformoney).text
    except:
        pass

    try:
        guests = driver.find_element(By.XPATH, XPATH_GUESTS).text
        bedrooms = driver.find_element(By.XPATH, XPATH_BEDROOMS).text
        beds = driver.find_element(By.XPATH, XPATH_BEDS).text
        bathrooms = driver.find_element(By.XPATH, XPATH_BATHROOMS).text
    except:
        pass

    apartment_ameneties = scrape_ameneties(driver)
    comments = scrape_comments(driver)
    lat, lng = scrape_coordinates(driver)

    flat = {'name': [apartment_name], 'price': [apartment_price], 'price_reduced': [apartment_price_reduced], 'place': [apartment_place], 'guests': [guests], 'bedrooms': [bedrooms], 'beds': [beds], 'bathrooms': [bathrooms], 'lat': [lat], 'lng': [lng], 'nr_of_ratings': [nr_of_ratings], 'rating_overall': [rating], 'rating_valueformoney': [rating_valueformoney],
            'rating_location': [rating_location], 'rating_checkin': [rating_checkin], 'rating_communication': [rating_communication], 'rating_cleanliness': [rating_cleanliness], 'rating_data_accuracy': [rating_data_accuracy], 'comments': [comments], 'list_ameneties': [apartment_ameneties]}

    print(nr_of_flat)

    return flat


In [12]:
def close_tab(driver):
    try:
        if len(driver.window_handles) > 1:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(1)
    except:
        print("could not close tab")

In [13]:
# helper func to open each aparement in sepeerate tab
def handle_apartments_tabs(nr_apartments, driver):
    # Click on each apartment which is opened in seperate tab
    apartments = pd.DataFrame()
    for nr in range(1, nr_apartments):
        xpath = f"/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div[{nr}]"
        driver.find_element(By.XPATH, xpath).click()

        try:
            time.sleep(1)
            # switch to new tab
            driver.switch_to.window(driver.window_handles[-1])
        except:
            print("could not switch tab")
            continue

        try:
            # close translate banner
            transl = WebDriverWait(driver, 5).until(EC.element_to_be_clickable(
                (By.XPATH, "//button[@aria-label='Schließen']")))
            transl.click()
        except:
            pass

        apartment_scraped = scrape_apartment(nr, driver)

        if apartment_scraped == None:
            close_tab(driver)
            continue

        apartments = pd.concat([apartments, pd.DataFrame(apartment_scraped)])
        close_tab(driver)

    return apartments


# Run scrape logic

In [14]:
url_to_scrape = buildURL(url_range)
url_to_scrape

['https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=0&price_max=68',
 'https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=69&price_max=84',
 'https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B

In [15]:
# run headless
# Set options for Chrome-Driver (headless mode)
opts = Options()
#opts.add_argument("--window-size=1200,800")
opts.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# Website to scrape
driver.get(url_to_scrape[6])

# Get current url
print(driver.current_url)

# Return the full page HTML code
# print(driver.page_source)

# Gets the page's title
print(driver.title)

# Close driver
# driver.close()

https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=151&price_max=173
Erstklassige Unterkünfte & Ferienunterkünfte in Zürich: Gesamte Unterkunft | Airbnb


In [16]:
# wait till cookie banner pops up
try:
    # close cookie banner
    cookie = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html[@class='js-focus-visible']/body[@class='with-new-header']/div[5]/div/div/div/div[1]/div[3]/section/div[2]/div[2]/button"))) 
    cookie.click()
except:
    pass

In [17]:
# get nr of pages with aparements
nav_buttons_pages = driver.find_elements(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[3]/div/div/nav/div/*")
# get nr of pages
nr_pages = int(nav_buttons_pages[(len(nav_buttons_pages)-2)].text)

In [18]:
nr_pages

11

In [19]:
# get entries of apparements on page
nr_apartments = len(driver.find_elements(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div"))

In [20]:
nr_apartments

18

In [21]:
# all entries
# scraped_apartments = pd.DataFrame()
# XPATH_NEXT_PAGE = "//a[@aria-label='Weiter']"
# XPATH_WAIT_TILL_ELEM_VISIBLE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div[1]"

# for nr_p in range(1, nr_pages):
#     WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, XPATH_WAIT_TILL_ELEM_VISIBLE)))
#     temp_pd = handle_apartments_tabs(nr_apartments + 1, driver)
#     scraped_apartments = pd.concat([scraped_apartments, temp_pd])
#     driver.find_element(By.XPATH, XPATH_NEXT_PAGE).click()

In [18]:
nr_pages = 2
handle_apa = 3

# limited entries for tests 
scraped_apartments = pd.DataFrame()
XPATH_NEXT_PAGE = "//a[@aria-label='Weiter']"
XPATH_WAIT_TILL_ELEM_VISIBLE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div[1]"

for nr_p in range(1, nr_pages):
    WebDriverWait(driver, 4).until(EC.visibility_of_element_located((By.XPATH, XPATH_WAIT_TILL_ELEM_VISIBLE)))
    temp_pd = handle_apartments_tabs(handle_apa, driver)
    scraped_apartments = pd.concat([scraped_apartments, temp_pd])
    driver.find_element(By.XPATH, XPATH_NEXT_PAGE).click()

1 {'name': ['Gemütliche & zentrale Wohnung in der Stadt Zürich'], 'price': ['145 CHF '], 'price_reduced': [''], 'place': ['Zürich, Schweiz'], 'guests': ['3 Gäste'], 'bedrooms': ['1 Schlafzimmer'], 'beds': ['1 Bett'], 'bathrooms': ['1 Badezimmer'], 'lat': [47.391], 'lng': [8.4785], 'nr_of_ratings': ['12 Bewertungen'], 'rating_overall': ['4,92 ·'], 'rating_valueformoney': ['4,9'], 'rating_location': ['4,8'], 'rating_checkin': ['5,0'], 'rating_communication': ['5,0'], 'rating_cleanliness': ['5,0'], 'rating_data_accuracy': ['5,0'], 'comments': [['Sehr ein netter freundlicher Gastgeber kann ich nur weiter empfehlen zudem ist er sehr flexibel\n\nDie Wohnung ansich war sehr sauber … 1:1 wie auf den Bildern - man kann direkt auch zu Fuss gleich zum Migros… von der Lage her auch super alles schnell erreichbar für den Preis ist das ein Geschenk danke nochmals !', 'Very cozy and bright apartment, has everything even for a long stay. Located in a fairly quiet area.\nThe public transport stop is lo

In [19]:
scraped_apartments.head()

,name,price,price_reduced,place,guests,bedrooms,beds,bathrooms,lat,lng,nr_of_ratings,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,comments,list_ameneties
0,Gemütliche & zentrale Wohnung in der Stadt Zürich,145 CHF,,"Zürich, Schweiz",3 Gäste,1 Schlafzimmer,1 Bett,1 Badezimmer,47.39100,8.47850,12 Bewertungen,"4,92 ·","4,9","4,8","5,0","5,0","5,0","5,0",[Sehr ein netter freundlicher Gastgeber kann i...,"[Föhn, Reinigungsprodukte, Shampoo der Marke K..."
0,Family big Royal Suite mit Balkon 4.3.,132 CHF,,"Zürich, Schweiz",4 Gäste,1 Schlafzimmer,2 Betten,1 Badezimmer,47.35924,8.55265,98 Bewertungen,"4,81 ·","4,7","4,9","4,9","4,8","4,7","4,8",[War früher immer im Viertel Seefeld im Hotel ...,"[Föhn, Reinigungsprodukte, Shampoo, Seife oder..."


In [22]:
# close all tabs and window
for handles in driver.window_handles:
    parent = handles
    driver.switch_to.window(parent)
    driver.close()

In [25]:
scraped_apartments.to_csv("./airbnb_apartments_scraped.csv", encoding="utf-8", index=False, header=True, errors="replace")

# Preparation to run scrape logic in parallel

In [23]:
def run_scraping(airbnb_url_to_scrape) -> DataFrame:
    opts = Options()
    #opts.add_argument("--window-size=1200,800")
    opts.add_argument("--headless")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    driver.get(airbnb_url_to_scrape)

    time.sleep(2)

    # wait till cookie banner pops up
    try:
        # close cookie banner
        cookie = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html[@class='js-focus-visible']/body[@class='with-new-header']/div[5]/div/div/div/div[1]/div[3]/section/div[2]/div[2]/button"))) 
        cookie.click()
    except:
        pass

    # get nr of pages with aparements
    nav_buttons_pages = driver.find_elements(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[3]/div/div/nav/div/*")
    try:
        # get nr of pages
        nr_pages = int(nav_buttons_pages[(len(nav_buttons_pages)-2)].text)
    except:
        nr_pages = 1
    
    # get entries of apparements on page
    nr_apartments = len(driver.find_elements(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div"))

    # limited entries for tests 
    scraped_apartments = pd.DataFrame()
    XPATH_NEXT_PAGE = "//a[@aria-label='Weiter']"
    XPATH_WAIT_TILL_ELEM_VISIBLE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div[1]"

    for nr_p in range(1, nr_pages):
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, XPATH_WAIT_TILL_ELEM_VISIBLE)))
        temp_pd = handle_apartments_tabs(nr_apartments + 1, driver)
        scraped_apartments = pd.concat([scraped_apartments, temp_pd])
        driver.find_element(By.XPATH, XPATH_NEXT_PAGE).click()
    
    # close all tabs and window
    for handles in driver.window_handles:
        parent = handles
        driver.switch_to.window(parent)
        driver.close()

    return scraped_apartments

In [51]:
# scrape all entries single threaded
scraped_apartments = pd.DataFrame()
url_to_scrape = buildURL(url_range)

for url_scrape in url_to_scrape:
    result = run_scraping(url_scrape)
    scraped_apartments = pd.concat([scraped_apartments, result])

scraped_apartments.to_csv("../data/airbnb_apartments_scraped.csv", encoding="utf-8", index=False, header=True, errors="replace")

KeyboardInterrupt: 

# Run scrape logic in parallel

In [24]:
import time
import multiprocessing as mp
import multiprocessing_func

scraped_apartments = pd.DataFrame()
url_to_scrape = buildURL(url_range)

with mp.Pool(2) as pool:
    result = pool.map(multiprocessing_func.run_scraping, url_to_scrape)

for res in result:
    scraped_apartments = pd.concat([scraped_apartments, res])

scraped_apartments.to_csv("../data/airbnb_apartments_scraped.csv", encoding="utf-8", index=False, header=True, errors="replace")

1
1
2
2
3
3
4
5
6
8
9
10
11
Timed out waiting for page to load
14
15
17
Timed out waiting for page to load
could not process comments
2
3
5
6
7
1
9
3
4
12
5
13
6
14
7
8
9
10
1
11
3
12
4
13
5
14
6
16
7
17
8
18
9
10
could not process comments
1
11
2
12
3
13
4
14
5
15
6
16
7
17
8
18
9
10
Timed out waiting for page to load
2
11
3
12
4
13
5
14
6
15
7
16
8
17
9
18
10
Timed out waiting for page to load
11
2
12
3
13
4
14
5
6
16
7
17
8
18
9
Timed out waiting for page to load
10
2
11
3
12
4
13
5
14
6
15
7
16
8
17
could not process comments
9
18
could not process comments
Timed out waiting for page to load
11
2
12
13
4
14
5
15
6
7
17
8
18
9
Timed out waiting for page to load
10
2
11
3
12
13
6
14
7
8
17
10
11
Timed out waiting for page to load
12
2
3
14
5
15
7
8
1
9
2
10
3
4
12
5
13
6
7
15
could not process comments
8
16
9
10
18
11
Timed out waiting for page to load
12
2
13
3
14
4
could not process comments
15
5
16
6
17
18
8
Timed out waiting for page to load
9
2
10
could not process comments
3
11